# Week 7: Python for the Web - Backend Basics with Flask
## Building APIs and Full-Stack Applications

This notebook demonstrates Flask fundamentals, building RESTful APIs, database integration, and full-stack development patterns.

## 1. Flask Basics - Creating a Simple Server

In [ ]:
# Installation
# pip install flask flask-cors python-dotenv

from flask import Flask, jsonify, request
from flask_cors import CORS
import json
from datetime import datetime

# Create Flask application
app = Flask(__name__)
CORS(app)  # Enable CORS for frontend communication

# Basic route - GET
@app.route('/api/hello', methods=['GET'])
def hello():
    """
    Simple GET endpoint that returns a greeting
    """
    return jsonify({
        'message': 'Hello from Flask!',
        'timestamp': datetime.now().isoformat()
    }), 200

# Route with URL parameter
@app.route('/api/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    """
    GET endpoint with URL parameter
    Example: /api/users/123
    """
    user = {
        'id': user_id,
        'name': 'John Doe',
        'email': 'john@example.com'
    }
    return jsonify(user), 200

# Route with query parameters
@app.route('/api/search', methods=['GET'])
def search():
    """
    GET endpoint with query parameters
    Example: /api/search?q=python&limit=10
    """
    query = request.args.get('q', 'default')
    limit = request.args.get('limit', 5, type=int)
    
    return jsonify({
        'query': query,
        'limit': limit,
        'results': ['result1', 'result2']
    }), 200

# Run server
# if __name__ == '__main__':
#     app.run(debug=True, port=5000)

## 2. RESTful API - POST, PUT, DELETE Operations

In [ ]:
# In-memory storage for demo (use database in production)
users_db = {
    1: {'id': 1, 'name': 'Alice', 'email': 'alice@example.com'},
    2: {'id': 2, 'name': 'Bob', 'email': 'bob@example.com'}
}
next_user_id = 3

# GET all users
@app.route('/api/users', methods=['GET'])
def get_all_users():
    return jsonify(list(users_db.values())), 200

# POST - Create new user
@app.route('/api/users', methods=['POST'])
def create_user():
    """
    POST endpoint to create a new user
    Expected JSON: {"name": "Jane", "email": "jane@example.com"}
    """
    global next_user_id
    
    # Get JSON data
    data = request.get_json()
    
    # Validate required fields
    if not data:
        return jsonify({'error': 'No data provided'}), 400
    
    if 'name' not in data or not data['name']:
        return jsonify({'error': 'Name is required'}), 400
    
    if 'email' not in data or not data['email']:
        return jsonify({'error': 'Email is required'}), 400
    
    # Create user
    new_user = {
        'id': next_user_id,
        'name': data['name'],
        'email': data['email']
    }
    
    users_db[next_user_id] = new_user
    next_user_id += 1
    
    return jsonify(new_user), 201  # 201 = Created

# PUT - Update user
@app.route('/api/users/<int:user_id>', methods=['PUT'])
def update_user(user_id):
    """
    PUT endpoint to update a user
    """
    if user_id not in users_db:
        return jsonify({'error': 'User not found'}), 404
    
    data = request.get_json()
    
    # Update fields if provided
    if 'name' in data:
        users_db[user_id]['name'] = data['name']
    if 'email' in data:
        users_db[user_id]['email'] = data['email']
    
    return jsonify(users_db[user_id]), 200

# DELETE - Remove user
@app.route('/api/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    """
    DELETE endpoint to remove a user
    """
    if user_id not in users_db:
        return jsonify({'error': 'User not found'}), 404
    
    deleted_user = users_db.pop(user_id)
    
    return jsonify({
        'message': 'User deleted successfully',
        'deleted_user': deleted_user
    }), 200

## 3. Error Handling - Status Codes and Error Responses

In [ ]:
# HTTP Status Codes
# 200 = OK (successful request)
# 201 = Created (resource created)
# 204 = No Content (successful, no data to return)
# 400 = Bad Request (client error - invalid data)
# 401 = Unauthorized (authentication required)
# 403 = Forbidden (permission denied)
# 404 = Not Found (resource doesn't exist)
# 422 = Unprocessable Entity (validation failed)
# 500 = Internal Server Error

@app.route('/api/validate', methods=['POST'])
def validate_data():
    """
    Example of error handling and validation
    """
    try:
        data = request.get_json()
        
        # Validate age
        age = data.get('age')
        if not isinstance(age, int) or age < 0 or age > 150:
            return jsonify({
                'error': 'Validation failed',
                'field': 'age',
                'message': 'Age must be a number between 0 and 150'
            }), 422
        
        return jsonify({
            'message': 'Validation successful',
            'data': data
        }), 200
        
    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'details': str(e)
        }), 500

## 4. Flask with SQLite Database Integration

In [ ]:
import sqlite3
from contextlib import contextmanager

DATABASE = 'tasks.db'

# Database connection helper
@contextmanager
def get_db():
    """Context manager for database connection"""
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    try:
        yield conn
    finally:
        conn.close()

def init_db():
    """Initialize database tables"""
    with get_db() as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS tasks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                description TEXT,
                completed BOOLEAN DEFAULT 0,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        conn.commit()

# GET all tasks
@app.route('/api/tasks', methods=['GET'])
def get_tasks():
    """Retrieve all tasks from database"""
    try:
        with get_db() as conn:
            cursor = conn.execute(
                'SELECT * FROM tasks ORDER BY created_at DESC'
            )
            tasks = [dict(row) for row in cursor.fetchall()]
            return jsonify(tasks), 200
    except sqlite3.Error as e:
        return jsonify({
            'error': 'Database error',
            'details': str(e)
        }), 500

# GET single task
@app.route('/api/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    """Retrieve a specific task"""
    try:
        with get_db() as conn:
            cursor = conn.execute(
                'SELECT * FROM tasks WHERE id = ?',
                (task_id,)
            )
            task = cursor.fetchone()
            
            if task is None:
                return jsonify({'error': 'Task not found'}), 404
            
            return jsonify(dict(task)), 200
    except sqlite3.Error as e:
        return jsonify({'error': 'Database error'}), 500

# POST - Create new task
@app.route('/api/tasks', methods=['POST'])
def create_task():
    """Create a new task"""
    try:
        data = request.get_json()
        
        # Validate required field
        if not data or 'title' not in data or not data['title']:
            return jsonify({'error': 'Title is required'}), 400
        
        with get_db() as conn:
            cursor = conn.execute(
                '''INSERT INTO tasks (title, description, completed)
                   VALUES (?, ?, ?)''',
                (
                    data['title'],
                    data.get('description', ''),
                    data.get('completed', False)
                )
            )
            conn.commit()
            
            return jsonify({
                'id': cursor.lastrowid,
                'title': data['title'],
                'description': data.get('description', ''),
                'completed': False
            }), 201
    except sqlite3.Error as e:
        return jsonify({'error': 'Database error'}), 500

# PUT - Update task
@app.route('/api/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    """Update an existing task"""
    try:
        data = request.get_json()
        
        with get_db() as conn:
            # Check if task exists
            cursor = conn.execute('SELECT id FROM tasks WHERE id = ?', (task_id,))
            if not cursor.fetchone():
                return jsonify({'error': 'Task not found'}), 404
            
            # Update task
            updates = []
            params = []
            
            if 'title' in data:
                updates.append('title = ?')
                params.append(data['title'])
            if 'description' in data:
                updates.append('description = ?')
                params.append(data['description'])
            if 'completed' in data:
                updates.append('completed = ?')
                params.append(data['completed'])
            
            if updates:
                updates.append('updated_at = CURRENT_TIMESTAMP')
                params.append(task_id)
                
                query = f'UPDATE tasks SET {" , ".join(updates)} WHERE id = ?'
                conn.execute(query, params)
                conn.commit()
            
            # Return updated task
            cursor = conn.execute('SELECT * FROM tasks WHERE id = ?', (task_id,))
            task = cursor.fetchone()
            return jsonify(dict(task)), 200
    except sqlite3.Error as e:
        return jsonify({'error': 'Database error'}), 500

# DELETE - Remove task
@app.route('/api/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    """Delete a task"""
    try:
        with get_db() as conn:
            cursor = conn.execute('SELECT id FROM tasks WHERE id = ?', (task_id,))
            if not cursor.fetchone():
                return jsonify({'error': 'Task not found'}), 404
            
            conn.execute('DELETE FROM tasks WHERE id = ?', (task_id,))
            conn.commit()
            
            return jsonify({'message': 'Task deleted successfully'}), 200
    except sqlite3.Error as e:
        return jsonify({'error': 'Database error'}), 500

## 5. Working with CSV/JSON Files for Data Persistence

In [ ]:
import csv
import os
from pathlib import Path

# File-based storage helper
class FileDataStore:
    """Simple file-based data storage using JSON"""
    
    def __init__(self, filename):
        self.filename = filename
        self.ensure_file_exists()
    
    def ensure_file_exists(self):
        """Create file if it doesn't exist"""
        if not os.path.exists(self.filename):
            with open(self.filename, 'w') as f:
                json.dump([], f)
    
    def read_all(self):
        """Read all records"""
        try:
            with open(self.filename, 'r') as f:
                return json.load(f)
        except (json.JSONDecodeError, FileNotFoundError):
            return []
    
    def write_all(self, data):
        """Write all records"""
        with open(self.filename, 'w') as f:
            json.dump(data, f, indent=2)
    
    def get_by_id(self, record_id):
        """Get single record by ID"""
        records = self.read_all()
        return next((r for r in records if r['id'] == record_id), None)
    
    def add(self, record):
        """Add new record"""
        records = self.read_all()
        record['id'] = max([r['id'] for r in records], default=0) + 1
        records.append(record)
        self.write_all(records)
        return record
    
    def update(self, record_id, updates):
        """Update record"""
        records = self.read_all()
        for record in records:
            if record['id'] == record_id:
                record.update(updates)
                self.write_all(records)
                return record
        return None
    
    def delete(self, record_id):
        """Delete record"""
        records = self.read_all()
        filtered = [r for r in records if r['id'] != record_id]
        self.write_all(filtered)
        return len(filtered) < len(records)

# Example: Recipe storage
recipes_store = FileDataStore('recipes.json')

@app.route('/api/recipes', methods=['GET'])
def get_recipes():
    recipes = recipes_store.read_all()
    return jsonify(recipes), 200

@app.route('/api/recipes', methods=['POST'])
def create_recipe():
    data = request.get_json()
    
    if not data or 'name' not in data:
        return jsonify({'error': 'Recipe name is required'}), 400
    
    recipe = recipes_store.add({
        'name': data['name'],
        'ingredients': data.get('ingredients', []),
        'instructions': data.get('instructions', '')
    })
    
    return jsonify(recipe), 201

@app.route('/api/recipes/<int:recipe_id>', methods=['DELETE'])
def delete_recipe(recipe_id):
    if recipes_store.delete(recipe_id):
        return jsonify({'message': 'Recipe deleted'}), 200
    return jsonify({'error': 'Recipe not found'}), 404

## 6. Configuration and Environment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

class Config:
    """Base configuration"""
    SECRET_KEY = os.getenv('SECRET_KEY', 'dev-secret-key')
    DEBUG = os.getenv('DEBUG', 'False') == 'True'
    DATABASE = os.getenv('DATABASE', 'app.db')
    FLASK_PORT = os.getenv('FLASK_PORT', 5000)

class DevelopmentConfig(Config):
    """Development configuration"""
    DEBUG = True
    TESTING = False

class ProductionConfig(Config):
    """Production configuration"""
    DEBUG = False
    TESTING = False

class TestingConfig(Config):
    """Testing configuration"""
    TESTING = True
    DATABASE = ':memory:'

# Apply configuration
config = os.getenv('FLASK_ENV', 'development')
if config == 'production':
    app.config.from_object(ProductionConfig)
else:
    app.config.from_object(DevelopmentConfig)

# Example .env file content:
# SECRET_KEY=your-super-secret-key-here
# DATABASE=tasks.db
# DEBUG=True
# FLASK_PORT=5000
# FLASK_ENV=development

## 7. Request Validation and Error Handling

In [ ]:
import re

class ValidationError(Exception):
    """Custom validation error"""
    def __init__(self, message, field=None):
        self.message = message
        self.field = field
        super().__init__(self.message)

def validate_email(email):
    """Validate email format"""
    pattern = r'^[^\s@]+@[^\s@]+\.[^\s@]+$'
    return re.match(pattern, email) is not None

def validate_user_data(data):
    """Validate user input data"""
    errors = {}
    
    # Name validation
    name = data.get('name', '').strip()
    if not name:
        errors['name'] = 'Name is required'
    elif len(name) < 2:
        errors['name'] = 'Name must be at least 2 characters'
    elif len(name) > 100:
        errors['name'] = 'Name must not exceed 100 characters'
    
    # Email validation
    email = data.get('email', '').strip()
    if not email:
        errors['email'] = 'Email is required'
    elif not validate_email(email):
        errors['email'] = 'Invalid email format'
    
    # Age validation
    if 'age' in data:
        try:
            age = int(data['age'])
            if age < 0 or age > 150:
                errors['age'] = 'Age must be between 0 and 150'
        except ValueError:
            errors['age'] = 'Age must be a number'
    
    return errors

@app.route('/api/users/validate', methods=['POST'])
def validate_user():
    """Endpoint to validate user data"""
    try:
        data = request.get_json()
        
        if not data:
            return jsonify({'error': 'No data provided'}), 400
        
        errors = validate_user_data(data)
        
        if errors:
            return jsonify({
                'valid': False,
                'errors': errors
            }), 422
        
        return jsonify({
            'valid': True,
            'message': 'All data is valid'
        }), 200
    
    except Exception as e:
        return jsonify({
            'error': 'Validation error',
            'details': str(e)
        }), 500

## 8. CORS (Cross-Origin Resource Sharing) Configuration

In [ ]:
# CORS allows your frontend (different domain) to access your backend

# Enable CORS globally (already done with CORS(app))
# This allows requests from any domain

# For more specific CORS configuration:
from flask_cors import cross_origin

# CORS for specific routes
@app.route('/api/public', methods=['GET'])
@cross_origin()  # Enable CORS for this route
def public_endpoint():
    return jsonify({'data': 'public data'}), 200

# Configure CORS with specific options
# CORS(app, resources={
#     r"/api/*": {
#         "origins": ["http://localhost:3000", "https://yourdomain.com"],
#         "methods": ["GET", "POST", "PUT", "DELETE"],
#         "allow_headers": ["Content-Type", "Authorization"]
#     }
# })

## 9. Complete Example: Task Manager API with Database

In [ ]:
# app.py - Complete working example

from flask import Flask, jsonify, request
from flask_cors import CORS
import sqlite3
from datetime import datetime

app = Flask(__name__)
CORS(app)

DATABASE = 'tasks.db'

def get_db():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    with app.app_context():
        db = get_db()
        db.execute('''
            CREATE TABLE IF NOT EXISTS tasks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                description TEXT,
                completed BOOLEAN DEFAULT 0,
                priority TEXT DEFAULT 'medium',
                due_date TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        db.commit()
        db.close()

# Health check endpoint
@app.route('/api/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat()
    }), 200

# Get all tasks with filtering
@app.route('/api/tasks', methods=['GET'])
def get_tasks_all():
    status = request.args.get('status', 'all')  # all, pending, completed
    priority = request.args.get('priority')  # optional filter
    
    try:
        db = get_db()
        query = 'SELECT * FROM tasks'
        params = []
        
        if status == 'pending':
            query += ' WHERE completed = 0'
        elif status == 'completed':
            query += ' WHERE completed = 1'
        
        if priority:
            if params:
                query += f" AND priority = '{priority}'"
            else:
                query += f" WHERE priority = '{priority}'"
        
        query += ' ORDER BY created_at DESC'
        
        cursor = db.execute(query)
        tasks = [dict(row) for row in cursor.fetchall()]
        db.close()
        
        return jsonify(tasks), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the app
if __name__ == '__main__':
    init_db()
    # Flask will run on http://localhost:5000
    # app.run(debug=True, port=5000)
    print("Flask server example created. See week7_javascript_examples.js for frontend code.")

## 10. Testing Your API

In [ ]:
# Using Python's requests library to test your API
# pip install requests

import requests

BASE_URL = 'http://localhost:5000/api'

# Test GET all tasks
# response = requests.get(f'{BASE_URL}/tasks')
# print('GET /tasks:', response.status_code, response.json())

# Test POST new task
# new_task = {
#     'title': 'Learn Flask',
#     'description': 'Build web applications with Python',
#     'priority': 'high'
# }
# response = requests.post(f'{BASE_URL}/tasks', json=new_task)
# print('POST /tasks:', response.status_code, response.json())

# Test PUT update
# updated = {'completed': True}
# response = requests.put(f'{BASE_URL}/tasks/1', json=updated)
# print('PUT /tasks/1:', response.status_code, response.json())

# Test DELETE
# response = requests.delete(f'{BASE_URL}/tasks/1')
# print('DELETE /tasks/1:', response.status_code, response.json())

# Common testing tools:
# - Postman: GUI tool for testing APIs
# - Thunder Client: VS Code extension
# - curl: Command line tool
# - pytest: Python testing framework